In [1]:
from new_SiameseNet import *

Using Theano backend.
/Users/wilbury/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [3]:
# Prepare data
print 'Getting CIFAR10 data...'

data = get_CIFAR10_data()

x_train, y_train = data['X_train'], data['y_train']
x_val,   y_val   = data['X_val'],   data['y_val']

N = x_train.shape[0]

Getting CIFAR10 data...


In [5]:
# Specify structure of Siamese part of SiameseNet
# This part needs to be improved. I'm kind of just using random layers.
init = 'glorot_uniform'
in_shp = (3,32,32)
seq = Sequential()
seq.add(BatchNormalization(epsilon=1e-7,
                            mode=0,
                            axis=1,
                            momentum=0.9,
                            weights=None,
                            input_shape=in_shp))
seq.add(Flatten())
seq.add(Dense(128, activation='relu'))
seq.add(Dropout(0.1))
seq.add(Dense(128, activation='relu'))
seq.add(Dropout(0.1))
seq.add(Dense(128, activation='relu'))
layers = seq

sn = SiameseNet(layers, input_shape=(3, 32, 32), verbose=True)
sn.compile()

Constructed a SiameseNet.
Successfully compiled the SiameseNet.


In [ ]:
sn.load(filepath='weights.h5')

In [7]:
def _train_sn(sn, x_train, y_train, filepath):
    d_train = invert_dataset(x_train,  y_train)
    sn.fit(*generate_data(d_train, examples_per_image=1)) #, validation_data=generate_data(x_val, y_val))
    #sn.save(filepath)

In [8]:
_train_sn(sn, x_train, y_train, filepath='weights.h5')

Generating data...
Done generating data
Epoch 1/20
19588/19588 [==============================] - 9s - loss: nan     
Epoch 2/20
19588/19588 [==============================] - 7s - loss: nan     
Epoch 3/20
19588/19588 [==============================] - 8s - loss: nan     
Epoch 4/20
19588/19588 [==============================] - 8s - loss: nan     
Epoch 5/20
19588/19588 [==============================] - 8s - loss: nan     
Epoch 6/20
 1152/19588 [>.............................] - ETA: 8s - loss: nan

KeyboardInterrupt: 

In [ ]:
d_val = invert_dataset(x_val,  y_val)
loss = sn.evaluate(*generate_data(d_val, examples_per_image=5))

In [ ]:
val_x_dat, val_y_dat = generate_data(d_val, examples_per_image=5)
prediction = sn.predict(val_x_dat)[SiameseNet.OUTPUT]

ret_preds = prediction
max_d = np.max(ret_preds)
min_d = np.min(ret_preds)
thresh = (max_d + min_d) / 2.0
preds = [0,0]
for i,p in enumerate(prediction):
    if ret_preds[i] > thresh:
        preds[1] += 1
    else:
        preds[0] += 1

print preds